In [6]:
import pandas as pd
import numpy as np

In [7]:
dataset_original = pd.read_csv('https://raw.githubusercontent.com/anaC-Fernandes/ProjetoDados/refs/heads/main/Datasets/BrStats.csv', sep = ";", low_memory=False)
df = dataset_original

In [8]:
# Tratamento da formatação dos dados
for coluna in df.columns:
    df[coluna] = df[coluna].astype(str)  # Garante que é string
    df[coluna] = df[coluna].str.replace('.', '', regex=False)  # Remove separador de milhar
    df[coluna] = df[coluna].str.replace(',', '.', regex=False)  # Converte decimal
    df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
df

for coluna in df.columns:
  df[coluna] = df[coluna].astype(str)  # Garante que é string
  df[coluna] = df[coluna].str.replace('nan', '0', regex=False)  # Converte decimal

df

# Tratamento de tipo

df['Ano'] = df['Ano'].astype(int)
df['CDMunicipio'] = df['CDMunicipio'].astype(str)
df['Populacao'] = df['Populacao'].astype(int)
df['PessoalOcupado'] = df['PessoalOcupado'].astype(int)
df['PessoalAssalariado'] = df['PessoalAssalariado'].astype(int)
df['VrSalarios'] = df['VrSalarios'].astype(int)
df['PIB'] = df['PIB'].astype(int)
df['QtEmpresas'] = df['QtEmpresas'].astype(int)
df['AreaPlantada_h'] = df['AreaPlantada_h'].astype(int)
df['AreaColhida_h'] = df['AreaColhida_h'].astype(int)
df['VlProducaoAgricola'] = df['VlProducaoAgricola'].astype(int)
df['VlProducaoPecuaria'] = df['VlProducaoPecuaria'].astype(int)
df['Area'] = df['Area'].astype(float)
df['Importacoes_US$'] = df['Importacoes_US$'].astype(float)
df['Exportacoes_US$'] = df['Exportacoes_US$'].astype(float)
df['Receitas_R$'] = df['Receitas_R$'].astype(float)
df['Transferencias_correntes_R$'] = df['Transferencias_correntes_R$'].astype(float)
df['Transferencias_capital_R$'] = df['Transferencias_capital_R$'].astype(float)
df['NrNascimentos'] = df['NrNascimentos'].astype(int)
df['NrObitosInfantis'] = df['NrObitosInfantis'].astype(int)
df['povoamento'] = df['povoamento'].astype(float)



C:\Users\anaca\AppData\Local\Temp\ipykernel_19092\374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
C:\Users\anaca\AppData\Local\Temp\ipykernel_19092\374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
C:\Users\anaca\AppData\Local\Temp\ipykernel_19092\374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
C:\Users\anaca\AppData

In [9]:
#tabela usada pra analisar os anos da Covid
tabela = df
tabela = tabela.drop(columns='AreaPlantada_h')
tabela = tabela.drop(columns='AreaColhida_h')
tabela = tabela.drop(columns='NrNascimentos')
tabela = tabela.drop(columns='NrObitosInfantis')
tabela = tabela.drop(columns='Area')
tabela = tabela.drop(columns='Importacoes_US$')
tabela = tabela.drop(columns='Exportacoes_US$')
tabela = tabela.drop(columns='Receitas_R$')
tabela = tabela.drop(columns='Transferencias_correntes_R$')
tabela = tabela.drop(columns='Transferencias_capital_R$')
tabela = tabela.drop(columns='VlProducaoAgricola')
tabela = tabela.drop(columns='VlProducaoPecuaria')

In [10]:
tabela['PIB_per_capita'] = tabela['PIB'] / tabela['Populacao'].replace(0, pd.NA)
tabela.sort_values(by='PIB_per_capita', ascending=False)


,Ano,CDMunicipio,Populacao,PessoalOcupado,PessoalAssalariado,VrSalarios,PIB,QtEmpresas,povoamento,UF,Municipio,Regiao,PIB_per_capita
10625,2020,1502152,38103,8553,7407,228990,22522725,9830,12.110,PA,Canaã dos Carajás,Norte,591.101094
15081,2018,3204302,11488,2730,2515,67664,6693537,1650,19.587,ES,Presidente Kennedy,Sudeste,582.654683
23853,2019,3204302,11574,2930,2669,70037,5383038,2000,19.734,ES,Presidente Kennedy,Sudeste,465.09746
8363,2019,3520400,34970,10333,8901,306489,14965490,1272,100.633,SP,Ilhabela,Sudeste,427.952245
6172,2018,3520400,34333,9378,8243,271697,14400916,9810,98.800,SP,Ilhabela,Sudeste,419.448228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32419,2017,4212650,0,0,0,0,0,0,0.000,SC,Pescaria Brava,Sul,<NA>
32537,2020,1504752,0,0,0,0,0,0,0.000,PA,Mojui dos Campos,Norte,<NA>
32691,2017,4314548,0,0,0,0,0,0,0.000,RS,Pinto Bandeira,Sul,<NA>
32723,2016,4220000,0,0,0,0,0,0,0.000,SC,Balneario Rincao,Sul,<NA>
